In [1]:
import numpy as np
from astropy.coordinates import CartesianRepresentation, GCRS, ITRS
from astropy.coordinates import EarthLocation, AltAz, get_body
from astropy.coordinates import Angle
from astropy.coordinates import EarthLocation
from astropy.coordinates import Longitude
from astropy.time import Time
import astropy.units as u

In [2]:
## Q1 ##
## ECEF to LatLon ##

## load data ##

r_vec = np.array([6789, 6893, 7035])

In [3]:
from latlon import ecef2latlon

phi, landa, h_ellp = ecef2latlon(r_vec)

print(phi*180/np.pi, landa*180/np.pi, h_ellp)

36.119599461442455 45.435509889654405 5591.508677843313


In [5]:
## part b ##
# ITRF to GCRF ##

# Define the ITRF position vector in km
r_itrf_km = [6789, 6893, 7035]

# Define the time of the observation
t = Time('2023-05-02T00:00:00', scale='utc')

# Convert the ITRF position vector to a CartesianRepresentation object
r_itrf = CartesianRepresentation(r_itrf_km, unit='km')

# Create an ITRS object with the ITRF position vector and time
itrs = ITRS(r_itrf, obstime=t)

# Convert the ITRS object to a GCRS object
gcrs = itrs.transform_to(GCRS(obstime=t))

# Extract the GCRF position vector in km
r_gcrf_km = gcrs.cartesian.xyz.value

print("Position vector in GCRF:", r_gcrf_km)


Position vector in GCRF: [ -862.5395007  -9634.74706215  7037.24872788]


In [6]:
## part c ##
## GMST and LST ##

# Define the longitude of the observation location in degrees
lon = Longitude('37.0', unit='deg')

# Define the location of the observation on Earth's surface
loc = EarthLocation(lon=lon, lat=0.0*u.deg, height=0.0*u.m)

# Define the time of the observation
t = Time('2023-04-30T14:30:00', scale='utc', location=loc)

# Calculate the GMST at the specified time
gmst = t.sidereal_time('mean')

# Convert the GMST to degrees
theta_GMST = gmst.to_value(u.deg)

# Calculate the local sidereal time (LST) at the specified location and time
lst = Angle(theta_GMST, unit='deg') + loc.lon

# Convert the LST to degrees
theta_LST = lst.to_value(u.deg)

print("GMST:", theta_GMST)
print("LST:", theta_LST)


GMST: 112.77872258804781
LST: 149.7787225880478


In [7]:
from skyfield.api import Topos, load

# Load the JPL ephemeris for the solar system bodies
eph = load('de421.bsp')

# Define a Topos object for the ground station
ground_topos = Topos(latitude_degrees=lat, longitude_degrees=lon)

# Define the start and end times for the observation period
start_time = obs_time.utc
end_time = (obs_time + 2*24*3600).utc

# Define the time step for the observation period
time_step = 60  # seconds

# Initialize lists to store the times and elevation angles
times = []
elevations = []

# Loop over the time range, calculating the elevation angle of the ISS relative to the ground station at each time step
for t in range(start_time, end_time, time_step):
    # Convert the time to a Skyfield time object
    skyfield_time = load.timescale().utc(t)

    # Get the position of the ISS relative to the ground station
    iss_pos_relative = ground_topos.at(skyfield_time).observe(get_body_barycentric('ISS', skyfield_time)).apparent().position.au
    iss_pos_relative_km = iss_pos_relative * 149597870.7  # convert from AU to km

    # Calculate the elevation angle of the ISS relative to the ground station
    r = iss_pos_relative_km - ground_pos_ecef.xyz.value
    r_norm = np.linalg.norm(r)
    elev = np.arcsin(r[2] / r_norm)

    # Append


NameError: name 'lat' is not defined

In [12]:
# import library
import numpy as np
from astropy.coordinates import EarthLocation, AltAz, get_body
from astropy.coordinates import Angle
from astropy.coordinates import EarthLocation
from astropy.coordinates import Longitude
from astropy.time import Time



loc = EarthLocation(lon=lon, lat=0.0*u.deg, height=0.0*u.m)
obs_time = Time('2023-04-30T14:30:00', scale='utc', location=loc)


# Define the start and end times for the observation period
start_time = obs_time.utc
end_time = (obs_time + 2*24*3600).utc

# Define the time step for the observation period
time_step = 60  # seconds

# Define a range of latitudes and longitudes to search over
latitudes = np.arange(-90, 90, 0.1)
longitudes = np.arange(-180, 180, 0.1)

# Initialize a dictionary to store the total view time for each location
view_times = {}

# Loop over the latitude and longitude ranges, calculating the total view time for each location
for lat in latitudes:
    for lon in longitudes:
        # Define the EarthLocation object for the current location
        ground_loc = EarthLocation.from_geodetic(lon=lon, lat=lat, height=0)

        # Get the ECEF position vector of the ground station at the observation time
        ground_pos_ecef = ground_loc.get_gcrs(obstime=obs_time).transform_to(ITRS(obstime=obs_time)).cartesian

        # Initialize a variable to store the total view time for the current location
        total_view_time = 0

        # Loop over the time range, calculating the elevation angle of the ISS relative to the ground station at each time step
        for t in range(start_time, end_time, time_step):
            # Convert the time to a Skyfield time object
            skyfield_time = load.timescale().utc(t)

            # Get the position of the ISS relative to the ground station
            iss_pos_relative = ground_topos.at(skyfield_time).observe(get_body_barycentric('ISS', skyfield_time)).apparent().position.au
            iss_pos_relative_km = iss_pos_relative * 149597870.7  # convert from AU to km

            # Calculate the elevation angle of the ISS relative to the ground station
            r = iss_pos_relative_km - ground_pos_ecef.xyz.value
            r_norm = np.linalg.norm(r)
            elev = np.arcsin(r[2] / r_norm)

            # If the elevation angle is greater than zero, add the time step to the total view time
            if elev > 0:
                total_view_time += time_step

        # Store the total view time for the current location in the dictionary
        view_times[(lat, lon)] = total_view_time

# Find the location with the highest total view time
best_location = max(view_times, key=view_times.get)

print(f"The best location for the ground station is at latitude {best_location[0]:.2f} degrees and longitude {best_location[1]:.2f} degrees, with a total view time of {view_times[best_location]/3600:.2f} hours over the 48 hour period.")


/Users/Ali/Documents/BAI/Master/Advanced_orbital_mechanics/HW/HW2/Code/Q4/myenv/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


TypeError: 'Time' object cannot be interpreted as an integer